# EXPLORATORY DATA ANALYSIS

In [4]:
# import libraries
import pandas as pd
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

plt.style.use('ggplot')
%matplotlib inline

In [5]:
# load data
file = 'data/cancer.data'
df = pd.read_csv(file)

In [7]:
def data_exp():
    profile = ProfileReport(df, title='Breast Cancer Profiling Report', html={'style':{'full_width':True}})
    return profile.to_widgets()


In [ ]:
data_exp()